In [2]:
%matplotlib notebook
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
root_init = 'C:\\Users\\Chahat\\Desktop\\dl\\BRATS2015_Training\\BRATS2015_Training\\'

In [4]:
def fetchData(root_init):
    tums = ["HGG", "LGG"]
    X_train = np.array([])
    Y_train = np.array([])
    i=0;j=0
    for tum in tqdm(tums):
        root = root_init + tum
        for person in tqdm(os.listdir(root)):
            indices = np.array([])
            #for test in os.listdir(root+"\\"+person):
            for test in os.listdir(os.path.join(root, person)):
                if "T1" in test and not "T1c" in test:
                    image_path = os.path.join(root, person, test, test + ".mha")
                    image_reader = sitk.ImageFileReader()
                    image_reader.SetImageIO("MetaImageIO")
                    image_reader.SetFileName(image_path)
                    image = image_reader.Execute()
                    image = sitk.GetArrayFromImage(image)
                    #image = image[:,40:215,30:205]
                    image = image[:,40:220,30:210]
                    #print(image.shape)

                    image_means = image.sum(axis = 1).sum(axis = 1) / (240 * 240)
                    indices = image_means>0
                    image = image[indices]
                    #print(image.shape)
                    #getting the maximum element of the 
                    max_array= np.amax(image, axis=1)
                    max_array= np.amax(max_array, axis=1)
                    max_array= np.reshape(max_array, (-1, 1, 1))
                    image= np.divide(image, max_array)
                    #image_means = image_means / image_means.max()
                    if i==0:
                        X_train= image
                        i=i+1
                    else:
                        X_train= np.concatenate((X_train, image), axis=0)

                if "OT" in test:
                    image_path = os.path.join(root, person, test, test + ".mha")
                    image_reader = sitk.ImageFileReader()
                    image_reader.SetImageIO("MetaImageIO")
                    image_reader.SetFileName(image_path)
                    gt = image_reader.Execute()
                    gt = sitk.GetArrayFromImage(gt)
                    gt = gt[indices]
                    gt = gt[:,40:215,30:205]
                    #print(gt.shape)
                    if j==0:
                        Y_train= gt
                        j=j+1
                    else:
                        Y_train= np.concatenate((Y_train, gt), axis=0)            


    S_train = Y_train.sum(axis = 1).sum(axis = 1) > 0 
    return X_train, Y_train, S_train

In [5]:
x,y,s = fetchData(root_init)

SystemError: <built-in function ImageFileReader_Execute> returned a result with an error set

In [55]:
print(x.shape)
print(y.shape)
print(s.shape)

(2701, 175, 175)
(2701, 175, 175)
(2701,)


In [56]:
root = "C:\\Users\\Chahat\\Desktop\\dl\\data"
np.save(os.path.join(root, "x_train.npy"), x)
np.save(os.path.join(root, "y_train.npy"), y)
np.save(os.path.join(root, "s_train.npy"), s)